In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
#import random
import numpy as np
import pandas as pd

In [52]:
from line_profiler import LineProfiler

In [4]:
import matplotlib.pyplot as plt

In [5]:
%matplotlib inline

In [6]:


#                             0    1+   31+   61+   91+   WOF
dod_migration = np.array([[0.95, 0.05, 0.00, 0.00, 0.00, 0.00], #  0 
                          [0.90, 0.05, 0.05, 0.00, 0.00, 0.00], #  1+
                          [0.10, 0.05, 0.05, 0.80, 0.00, 0.00], # 31+
                          [0.05, 0.05, 0.05, 0.05, 0.80, 0.00], # 61+
                          [0.01, 0.01, 0.02, 0.02, 0.04, 0.90], # 91+
                          [0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # WOF
                         ])
[i.sum() for i in dod_migration]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [7]:
class Contract():
    """Class Contract
       issue_dt - issue of contract
       duration - duration in months
    """

    dod_dic = {0: '0',
               1: '1+',
               2: '31+',
               3: '61+',
               4: '91+',
               5: 'WOF'
              }
    dod_cnt = 6 # кол-во состояний
    dod_states = np.eye(dod_cnt) # матрица состояний (для удобства использована единичная матрица)

    def __init__(self, cntr_id = 0, issue_dt = 0, duration = 0,
                 dod_migration = None):
        self.cntr_id = cntr_id
        self.dod_id = 0        # начальное состояние контракта при выдачи: DOD = 0
        self.dod_state = self.dod_states[0] # np.array([1,0,0,0,0]) 
        self.dod_migration = dod_migration
        self.issue_dt = issue_dt
        self.mob = 0
        self.duration = duration
        self.closed_id = 0       # 0 - контратк открыт, 1 - закрыт
        self.wrtoff_id = 0       # 0 - контратк несписан, 1 - списан
        
    def next_month(self):
        if self.closed_id == 1:
            return None
           
        self.mob = self.mob + 1
        p = self.dod_migration.T.dot(self.dod_state) # array of probabilities
        self.dod_id = np.random.choice(self.dod_cnt,1,p=p)[0] # new state
        self.dod_state = self.dod_states[self.dod_id]

        if self.dod_id == 0 and self.mob > self.duration: # погашение либо выздоровление с возвращением в график
            self.closed_id = 1
        
        if self.dod_id == 5 and self.mob > self.duration + 12: # списание
            self.wrtoff_id = 1

        if self.wrtoff_id == 1 and self.mob > self.duration + 24: # списание
            self.closed_id = 1
            

In [8]:
class World():
    """Class World - Макромир, который задает начало отсчета времени,
    законы макроэкономики, ограничения регуляторов/ЦБ и остальное окружение.
    """

    def __init__(self):
        self.World_Time = 0

In [144]:
class DWH_DB():
    """Class DWH - база данных
    """
    def __init__(self):
        self.LI = pd.DataFrame(columns = ['CNTR_ID',
                                          'SD',
                                          'DOD_ID',
                                          'MOB'
                                          ])
        self.DMContract = pd.DataFrame(columns = ['CNTR_ID',
                                                  'ISSUE_DT',
                                                  'WRITEOFF_DT',
                                                  'CLOSED_DT'
                                                 ])
        


In [116]:
class Portfolio():
    """Class Portfolio - Портфель - динамика 
        N - первая выдача при создании портфеля
        start_portfolio_dt - привязка портфеля к мировому времени - важно при наличии нескольких портфелей
    
    """
    def __init__(self, N = 10, start_portfolio_dt = 0):
        self.cntr_id = 0                                # счетчик контрактов
        self.start_portfolio_dt = start_portfolio_dt    # дата создания портфеля
        self.cntr_list = []                             # сам портфель
        self.portfolio_age = 0                          # возрвст портфеля

        # проведем первую выдачу - инициализация портфеля
        self.issue(N)
        # Заполним LI
        self.fix_in_dwh()

    def issue(self, N = 10):
        for i in range(N):
            self.cntr_id += 1
            self.cntr_list.append(Contract(cntr_id = self.cntr_id, issue_dt = self.start_portfolio_dt, duration = 36, dod_migration = dod_migration))

    def next_month(self, N = 10):
        self.portfolio_age +=1

        # сдвинем существующий портфель, потом проведем выдачу новых 
        test = []
        for cntr in self.cntr_list:
            if cntr.closed_id == 1:
                test.append(cntr.cntr_id)
                i = self.cntr_list.index(cntr)
                del self.cntr_list[i]
            else:
                cntr.next_month()
        print('%04i' % self.portfolio_age, len(self.cntr_list), 'out ->',  test)

        # проведем выдачи
        self.issue(N)

        # Заполним LI
        self.fix_in_dwh()

    def fix_in_dwh_old(self): # Пример медленной вставки
        ix = len(DWH.LI.index)
        for cnt in self.cntr_list:
            DWH.LI.loc[ix] = [cnt.cntr_id, self.portfolio_age, cnt.dod_id, cnt.mob]
            ix += 1

    def fix_in_dwh(self):
        DWH.LI = pd.concat([DWH.LI,
                            pd.DataFrame(data=[[cnt.cntr_id, self.portfolio_age, cnt.dod_id, cnt.mob] for cnt in self.cntr_list],
                                         columns=DWH.LI.columns)
                           ])
            
# journal
# tinkoff
# ru
# how-to-choose-yacht-school

In [145]:
%time
DWH = DWH_DB()
GP = Portfolio(50)

CPU times: user 3 μs, sys: 1e+03 ns, total: 4 μs
Wall time: 6.2 μs


In [146]:
for t in range(100):
    GP.next_month(50)

0001 50 out -> []
0002 100 out -> []
0003 150 out -> []
0004 200 out -> []
0005 250 out -> []
0006 300 out -> []
0007 350 out -> []
0008 400 out -> []
0009 450 out -> []
0010 500 out -> []
0011 550 out -> []
0012 600 out -> []
0013 650 out -> []
0014 700 out -> []
0015 750 out -> []
0016 800 out -> []
0017 850 out -> []
0018 900 out -> []
0019 950 out -> []
0020 1000 out -> []
0021 1050 out -> []
0022 1100 out -> []
0023 1150 out -> []
0024 1200 out -> []
0025 1250 out -> []
0026 1300 out -> []
0027 1350 out -> []
0028 1400 out -> []
0029 1450 out -> []
0030 1500 out -> []
0031 1550 out -> []
0032 1600 out -> []
0033 1650 out -> []
0034 1700 out -> []
0035 1750 out -> []
0036 1800 out -> []
0037 1850 out -> []
0038 1877 out -> [2, 4, 6, 9, 11, 13, 16, 18, 21, 23, 25, 27, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50]
0039 1890 out -> [3, 7, 10, 14, 17, 22, 26, 29, 33, 37, 41, 45, 49, 52, 54, 56, 58, 60, 62, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99]
0040 1898 

In [78]:
lp = LineProfiler()
lp_w = lp(GP.cntr_list[4].next_month)
lp_w() 

In [79]:
lp.print_stats()

Timer unit: 1e-09 s

Total time: 0.000848928 s
File: /tmp/ipykernel_41023/2834506441.py
Function: next_month at line 29

Line #      Hits         Time  Per Hit   % Time  Line Contents
    29                                               def next_month(self):
    30         1       3580.0   3580.0      0.4          if self.closed_id == 1:
    31                                                       return None
    32                                                      
    33         1       1420.0   1420.0      0.2          self.mob = self.mob + 1
    34         1     608179.0 608179.0     71.6          p = self.dod_migration.T.dot(self.dod_state) # array of probabilities
    35         1     230509.0 230509.0     27.2          self.dod_id = np.random.choice(self.dod_cnt,1,p=p)[0] # new state
    36         1       2480.0   2480.0      0.3          self.dod_state = self.dod_states[self.dod_id]
    37                                           
    38         1       1610.0   1610.0    

In [147]:
lp = LineProfiler()
lp_w = lp(GP.next_month)
for t in range(100):
    lp_w(50)

0101 1995 out -> [1613, 1730, 1810, 1854, 1894, 2941, 2982, 3028, 3049, 3060, 3068, 3076, 3089, 3099, 3103, 3107, 3111, 3115, 3119, 3122, 3126, 3130, 3134, 3138, 3142, 3146, 3150, 3153, 3155, 3157, 3159, 3162, 3164, 3166, 3168, 3170, 3173, 3175, 3177, 3179, 3181, 3183, 3185, 3189, 3191, 3195, 3197, 3199]
0102 1995 out -> [1832, 1933, 2997, 3009, 3036, 3053, 3064, 3080, 3101, 3109, 3117, 3124, 3136, 3144, 3151, 3154, 3158, 3161, 3165, 3169, 3174, 3178, 3182, 3186, 3190, 3196, 3200, 3203, 3205, 3207, 3209, 3211, 3213, 3215, 3217, 3219, 3221, 3223, 3225, 3227, 3229, 3231, 3233, 3235, 3237, 3241, 3243, 3245, 3247, 3249]
0103 1994 out -> [1948, 2005, 2029, 2046, 2087, 3072, 3093, 3105, 3121, 3132, 3148, 3156, 3163, 3171, 3176, 3184, 3192, 3201, 3204, 3212, 3216, 3220, 3224, 3228, 3232, 3236, 3242, 3248, 3251, 3253, 3256, 3258, 3260, 3262, 3264, 3266, 3268, 3270, 3273, 3275, 3277, 3279, 3281, 3283, 3285, 3288, 3290, 3292, 3294, 3297, 3299]
0104 1993 out -> [1708, 1888, 2000, 2016, 2082, 3113

In [148]:
lp.print_stats()

Timer unit: 1e-09 s

Total time: 9.59499 s
File: /tmp/ipykernel_41023/1999920913.py
Function: next_month at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents
    23                                               def next_month(self, N = 10):
    24       100     342024.0   3420.2      0.0          self.portfolio_age +=1
    25                                           
    26                                                   # сдвинем существующий портфель, потом проведем выдачу новых 
    27       100      47470.0    474.7      0.0          test = []
    28    198638   81392947.0    409.8      0.8          for cntr in self.cntr_list:
    29    198538   76958774.0    387.6      0.8              if cntr.closed_id == 1:
    30      5016    2682832.0    534.9      0.0                  test.append(cntr.cntr_id)
    31      5016    6383339.0   1272.6      0.1                  i = self.cntr_list.index(cntr)
    32      5016    3449353.0    687.7      0.0                 

In [151]:
T = DWH.LI.reset_index(drop=True)
T

,CNTR_ID,SD,DOD_ID,MOB
0,1,0,0,0
1,2,0,0,0
2,3,0,0,0
3,4,0,0,0
4,5,0,0,0
...,...,...,...,...
367307,10046,200,0,0
367308,10047,200,0,0
367309,10048,200,0,0
367310,10049,200,0,0


In [152]:
T[T['CNTR_ID']==30]

,CNTR_ID,SD,DOD_ID,MOB
29,30,0,0,0
79,30,1,0,1
179,30,2,0,2
329,30,3,0,3
529,30,4,0,4
779,30,5,0,5
1079,30,6,0,6
1429,30,7,0,7
1829,30,8,0,8
2279,30,9,0,9


In [112]:
G = T.groupby('SD')[['MOB']].agg(['count','mean'])
G

MOB           
    count       mean
SD                  
0      10        0.0
1      20        0.5
2      30        1.0
3      40        1.5
4      50        2.0
..    ...        ...
206   401  19.750623
207   399  19.616541
208   400    19.6875
209   399  19.656642
210   399  19.619048

[211 rows x 2 columns]

In [113]:
x = G.index
x

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       201, 202, 203, 204, 205, 206, 207, 208, 209, 210],
      dtype='int64', name='SD', length=211)

In [18]:
y = G['MOB']
y

SD
1       0.500000
2       1.000000
3       1.500000
4       2.000000
5       2.500000
         ...    
96     20.374083
97     20.463415
98     20.509756
99     20.545012
100    20.537713
Name: MOB, Length: 100, dtype: float64

In [17]:
cntr = Contract(cntr_id = 1, issue_dt = 0, duration = 36, dod_migration = dod_migration)
for i in range(cntr.duration + 10):
    cntr.next_month()
    print(cntr.mob, cntr.dod_state, cntr.closed_id, cntr.wrtoff_id)

1 [1. 0. 0. 0. 0. 0.] 0 0
2 [1. 0. 0. 0. 0. 0.] 0 0
3 [1. 0. 0. 0. 0. 0.] 0 0
4 [1. 0. 0. 0. 0. 0.] 0 0
5 [1. 0. 0. 0. 0. 0.] 0 0
6 [1. 0. 0. 0. 0. 0.] 0 0
7 [0. 1. 0. 0. 0. 0.] 0 0
8 [1. 0. 0. 0. 0. 0.] 0 0
9 [1. 0. 0. 0. 0. 0.] 0 0
10 [0. 1. 0. 0. 0. 0.] 0 0
11 [1. 0. 0. 0. 0. 0.] 0 0
12 [1. 0. 0. 0. 0. 0.] 0 0
13 [1. 0. 0. 0. 0. 0.] 0 0
14 [1. 0. 0. 0. 0. 0.] 0 0
15 [1. 0. 0. 0. 0. 0.] 0 0
16 [1. 0. 0. 0. 0. 0.] 0 0
17 [1. 0. 0. 0. 0. 0.] 0 0
18 [1. 0. 0. 0. 0. 0.] 0 0
19 [1. 0. 0. 0. 0. 0.] 0 0
20 [1. 0. 0. 0. 0. 0.] 0 0
21 [1. 0. 0. 0. 0. 0.] 0 0
22 [1. 0. 0. 0. 0. 0.] 0 0
23 [1. 0. 0. 0. 0. 0.] 0 0
24 [1. 0. 0. 0. 0. 0.] 0 0
25 [1. 0. 0. 0. 0. 0.] 0 0
26 [1. 0. 0. 0. 0. 0.] 0 0
27 [1. 0. 0. 0. 0. 0.] 0 0
28 [1. 0. 0. 0. 0. 0.] 0 0
29 [1. 0. 0. 0. 0. 0.] 0 0
30 [1. 0. 0. 0. 0. 0.] 0 0
31 [1. 0. 0. 0. 0. 0.] 0 0
32 [1. 0. 0. 0. 0. 0.] 0 0
33 [1. 0. 0. 0. 0. 0.] 0 0
34 [1. 0. 0. 0. 0. 0.] 0 0
35 [1. 0. 0. 0. 0. 0.] 0 0
36 [1. 0. 0. 0. 0. 0.] 0 0
37 [1. 0. 0. 0. 0. 0.] 1 0
37 [1. 0. 